#STEP 1

Add libraries and import the same.

In [ ]:
!pip install --upgrade tensorflow keras numpy pandas sklearn pillow split-folders wget fast_ml tensorboard matplotlib seaborn

In [ ]:
import numpy as np
import cv2 as cv
import glob

print (cv.__version__)


# STEP 2
Connect to Googel Drive or any other local storage

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "CameraCalibration"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  #!touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nYou can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)

In [ ]:
!pwd

In [ ]:
#Create the directories to store images and other data
os.mkdir ('images')
os.mkdir ('/images/camera_calibration_images') 

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/drive/My\ Drive/ /mydrive

In [ ]:
!ls /mydrive/CameraCalibration/

# STEP 3
Upload the chessboard images taken from the camera

In [ ]:
# Upload IMages

# STEP 4
CALIBRATE THE CAMERA TO OBTAIN THE PARAMETERS

In [ ]:
# Chess board squares and the frame width, height
chessboardSize = (7,7)
#chessboardSize = (8,9)
frameSize = (1088,1080)

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare the object points
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# Directory of images
# Local Path
#imagePath = 'C:/Users/parag/PycharmProjects/pythonProject1/images/camera_calibration_images/**/*.jpg'

# Cloud Path
imagePath = '/mydrive/CameraCalibration/images/camera_calibration_images/**/*.jpg'
images = glob.glob(imagePath, recursive=True)
print(images)

# DETECTION
for image in images:
    print(image)
    print('Reading image..')
    img = cv.imread(image)
    #print(img)
    print('Converting to GRAYSCALE')
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    print(gray)


    # Let us find the chessboard corners now
    ret, corners = cv.findChessboardCorners(gray, (7,7), None)
    #corners = np.array([[corner for [corner] in corners]])
    print(ret)
    #print (corners)

    # If objects are found and corners are returned, let us add the image and object points.
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (7, 7), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()


# CALIBRATION
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera Calibration Done:", ret)
print("\nCamera Matrix:\n", mtx)
print("\nDistortion Parameters:\n", dist)
print("\nRotation Vectors:\n", rvecs)
print("\nTranslation Vectors:\n", tvecs)


# UN-DISTORTION
img = cv.imread('G:\\My Drive\\SsdMobNet320\\data\\dataset\\val\\Glasses\\EyeGlasses916.jpg')
h,  w = img.shape[:2]
#newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))
#newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0.5, (w,h))

# UN-DISTORT IMAGES NOW
# METHOD-1
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('C:/Users/parag/PycharmProjects/pythonProject1/images/camera_calibration_images/calibresult.jpg', dst)

# METHOD-2
# undistort
#mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
#dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
"""# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]"""
#cv.imwrite('C:/Users/parag/PycharmProjects/pythonProject1/images/camera_calibration_images/calibresult.jpg', dst)

# RE-PROJECTION ERROR
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )